# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
file_path = r'C:\Users\sarit\OneDrive\Desktop\Class_Folder\mygithub\python-api-challenge\cities.csv'

city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,26.03,78,40,6.17,CK,1713183271
1,1,port-aux-francais,-49.3500,70.2167,4.77,93,99,13.36,TF,1713183271
2,2,arquitecto tomas romero pereira,-26.4833,-55.2500,22.52,96,100,3.50,PY,1713183272
3,3,santa ines,-3.6667,-45.3800,26.04,91,97,0.97,BR,1713183272
4,4,pimentel,-6.8367,-79.9342,20.16,83,0,7.72,PE,1713183273


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lat","Lng",geo = True,tiles = "OSM",frame_width=800,frame_height=600)


# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Define your ideal weather conditions
max_temp = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 20)
wind_speed = city_data_df['Wind Speed'] < 4.5
cloudiness = city_data_df['Cloudiness'] == 0

# Apply the filters to the DataFrame
ideal_weather_df = city_data_df[max_temp & wind_speed & cloudiness]

# Display the filtered DataFrame
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,villa vasquez,19.7411,-71.4464,24.60,72,0,2.83,DO,1713183319
189,189,archangelos,36.2149,28.1149,23.99,32,0,3.30,GR,1713183364
232,232,dakhla,23.6848,-15.9580,24.25,41,0,4.32,EH,1713183385
340,340,tazacorte,28.6290,-17.9293,21.54,83,0,3.60,ES,1713183438
385,385,selendi,38.7444,28.8678,26.07,20,0,1.58,TR,1713183461
389,389,balqash,46.8481,74.9950,20.33,28,0,4.00,KZ,1713183463
419,419,xi'an,34.2583,108.9286,21.69,31,0,1.74,CN,1713183478


In [32]:
ideal_weather_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.000000e+00
mean,292.714286,32.588657,19.367514,23.210000,43.857143,0.0,3.052857,1.713183e+09
std,122.137510,9.284212,60.784440,2.049203,24.009919,0.0,1.065031,6.004681e+01
min,95.000000,19.741100,-71.446400,20.330000,20.000000,0.0,1.580000,1.713183e+09
25%,210.500000,26.156900,-16.943650,21.615000,29.500000,0.0,2.285000,1.713183e+09
50%,340.000000,34.258300,28.114900,23.990000,32.000000,0.0,3.300000,1.713183e+09
75%,387.000000,37.479650,51.931400,24.425000,56.500000,0.0,3.800000,1.713183e+09
max,419.000000,46.848100,108.928600,26.070000,83.000000,0.0,4.320000,1.713183e+09


In [34]:
# Narrow down cities that fit criteria and drop any results with null values

# Drop any rows with null values
filtered_cities_df = ideal_weather_df.dropna()

filtered_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,villa vasquez,19.7411,-71.4464,24.60,72,0,2.83,DO,1713183319
189,189,archangelos,36.2149,28.1149,23.99,32,0,3.30,GR,1713183364
232,232,dakhla,23.6848,-15.9580,24.25,41,0,4.32,EH,1713183385
340,340,tazacorte,28.6290,-17.9293,21.54,83,0,3.60,ES,1713183438
385,385,selendi,38.7444,28.8678,26.07,20,0,1.58,TR,1713183461
389,389,balqash,46.8481,74.9950,20.33,28,0,4.00,KZ,1713183463
419,419,xi'an,34.2583,108.9286,21.69,31,0,1.74,CN,1713183478


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City','Country','Lat','Lng','Humidity']].copy()
               

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
95,villa vasquez,DO,19.7411,-71.4464,72,
189,archangelos,GR,36.2149,28.1149,32,
232,dakhla,EH,23.6848,-15.9580,41,
340,tazacorte,ES,28.6290,-17.9293,83,
385,selendi,TR,38.7444,28.8678,20,
389,balqash,KZ,46.8481,74.9950,28,
419,xi'an,CN,34.2583,108.9286,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [70]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # https://api.geoapify.com/v1/geocode/search?text=Berlin&lang=en&limit=10&type=city&apiKey=YOUR_API_KEY



    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df




Starting hotel search


villa vasquez - nearest hotel: No hotel found
archangelos - nearest hotel: Semina
dakhla - nearest hotel: Hotel El Massira
tazacorte - nearest hotel: App Leyma
selendi - nearest hotel: No hotel found
balqash - nearest hotel: No hotel found
xi'an - nearest hotel: Safiya Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
95,villa vasquez,DO,19.7411,-71.4464,72,No hotel found
189,archangelos,GR,36.2149,28.1149,32,Semina
232,dakhla,EH,23.6848,-15.9580,41,Hotel El Massira
340,tazacorte,ES,28.6290,-17.9293,83,App Leyma
385,selendi,TR,38.7444,28.8678,20,No hotel found
389,balqash,KZ,46.8481,74.9950,28,No hotel found
419,xi'an,CN,34.2583,108.9286,31,Safiya Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [95]:

for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    hotel_name = row["Hotel Name"]
    country = row["Country"]
    
     # Create a DataFrame with the marker information
    marker_df = pd.DataFrame({"Lat": [latitude], "Lng": [longitude], "info": [f"Hotel Name: {hotel_name}, Country: {country}"]})
    print(marker_df)
     # Configure the map plot with markers and hover information
     #map_plot_2 = marker_df.hvplot.points("Lat","Lng", hover_cols=["info"], geo=True, tiles="OSM", frame_width=800, frame_height=600)
    
    # Display the map
    # map_plot_2
   


       Lat      Lng                                     info
0  19.7411 -71.4464  Hotel Name: No hotel found, Country: DO
       Lat      Lng                             info
0  36.2149  28.1149  Hotel Name: Semina, Country: GR
       Lat     Lng                                       info
0  23.6848 -15.958  Hotel Name: Hotel El Massira, Country: EH
      Lat      Lng                                info
0  28.629 -17.9293  Hotel Name: App Leyma, Country: ES
       Lat      Lng                                     info
0  38.7444  28.8678  Hotel Name: No hotel found, Country: TR
       Lat     Lng                                     info
0  46.8481  74.995  Hotel Name: No hotel found, Country: KZ
       Lat       Lng                                   info
0  34.2583  108.9286  Hotel Name: Safiya Hotel, Country: CN
